In [1]:
from utils.BackTest import Engine, Strategy
import pandas as pd

In [2]:
df = pd.read_csv(r'../data/processed/combined_df_with_volatility.csv')
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

In [3]:
df

,datetime,reddit_smoothed_sentiment_weibull,news_smoothed_sentiment_weibull,tele_smoothed_sentiment_weibull,OPEN,HIGH,LOW,CLOSE,TOTAL_TRADES,TOTAL_TRADES_BUY,...,H-L,H-PC,L-PC,TR,ATR,log_return_100,predicted_conditional_volatility,rolling_mean_volatility,rolling_std_volatility,volatility_category
0,2024-03-23 18:00:00+00:00,-0.000662,232.686671,0.000000,65652.70,65681.72,65270.01,65373.88,56859,25413,...,411.71,29.02,382.69,411.71,658.445000,-0.425594,0.005914,0.006518,0.001199,Normal
1,2024-03-23 19:00:00+00:00,-0.000241,243.310915,0.000000,65373.88,65485.15,64786.25,64872.90,73300,32361,...,698.90,111.27,587.63,698.90,701.053333,-0.769282,0.005821,0.006418,0.001168,Normal
2,2024-03-23 20:00:00+00:00,-0.000083,263.874155,0.000000,64872.90,65118.20,64672.93,65009.78,55912,27474,...,445.27,245.30,199.97,445.27,604.058333,0.210775,0.006339,0.006296,0.001028,Normal
3,2024-03-23 21:00:00+00:00,-0.000027,265.004313,0.000000,65009.78,65199.97,64780.00,64944.69,45367,21035,...,419.97,190.19,229.78,419.97,571.245000,-0.100174,0.005951,0.006184,0.000926,Normal
4,2024-03-23 22:00:00+00:00,-0.000008,248.893742,0.000000,64944.69,65050.72,64744.55,64747.86,46105,21611,...,306.17,106.03,200.14,306.17,468.430000,-0.303533,0.005584,0.006049,0.000790,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,2025-01-26 07:00:00+00:00,0.055311,-13.564028,0.000000,104990.75,105100.00,104950.00,104998.35,41775,22248,...,150.00,109.25,40.75,150.00,298.715000,0.007238,0.003691,0.003776,0.000097,Normal
7406,2025-01-26 08:00:00+00:00,0.030398,-6.781369,0.103237,104998.35,105221.36,104863.31,105030.40,63281,30294,...,358.05,223.01,135.04,358.05,325.368333,0.030520,0.003626,0.003759,0.000091,Low
7407,2025-01-26 09:00:00+00:00,0.015620,0.681248,0.000000,105030.40,105030.41,104444.01,104630.03,91103,41647,...,586.40,0.01,586.39,586.40,326.935000,-0.381923,0.003572,0.003744,0.000097,Low
7408,2025-01-26 10:00:00+00:00,0.007520,8.190720,0.000000,104630.03,104722.90,104500.00,104715.29,50517,26960,...,222.90,92.87,130.03,222.90,277.221667,0.081454,0.003957,0.003754,0.000108,High


In [4]:
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

In [5]:
#Daily VWAP (UTC-based)
#Reset VWAP every UTC day. This is the most common standard for crypto backtests and analysis.
df['DATE'] = df.index.date  
df['VWAP'] = df.groupby('DATE').apply(
    lambda g: (g['CLOSE'] * g['VOLUME']).cumsum() / g['VOLUME'].cumsum()
).reset_index(level=0, drop=True)


# #Rolling VWAP (24-hour window)
# rolling_window = 24  # if hourly bars
# vwap_numerator = (df['CLOSE'] * df['VOLUME']).rolling(rolling_window).sum()
# vwap_denominator = df['VOLUME'].rolling(rolling_window).sum()
# df['VWAP'] = vwap_numerator / vwap_denominator

C:\Users\leoqi\AppData\Local\Temp\ipykernel_78724\947575711.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['VWAP'] = df.groupby('DATE').apply(


In [6]:
#VWAP strategy
def determine_signal(df):
    """
    Determine the trading signal based on the strategy's criteria.
    """
    if df['CLOSE'] > df['VWAP']:
        return 1
    elif df['CLOSE'] < df['VWAP']:
        return 0
    else:
        return None


In [7]:
df['VWAP_signal'] = df.apply(determine_signal, axis=1)

df['VWAP_signal'] = df['VWAP_signal'].fillna(0)

In [8]:
df_filtered = df.rename(columns={'OPEN':'Open',
                                 'CLOSE':'Close',
                                 'HIGH':'High',
                                 'LOW':'Low'})

In [9]:
class MultiStrategyAggregator(Strategy):
    def on_bar(self):
        if self.data.loc[self.current_idx]['VWAP_signal'] == 1:
            self.buy(
                "btc",
               size = round(0.10 * (self.cash + (self.position_size() * self.close())) / self.close(), 8)
#                size = round((0.10 * self.cash)/ self.close(), 8)
            )

In [10]:
validate_frame = df_filtered[df_filtered.index >= pd.to_datetime('2025-01-01').tz_localize('UTC')]

In [11]:
validate_frame.isna().sum()

reddit_smoothed_sentiment_weibull    0
news_smoothed_sentiment_weibull      0
tele_smoothed_sentiment_weibull      0
Open                                 0
High                                 0
Low                                  0
Close                                0
TOTAL_TRADES                         0
TOTAL_TRADES_BUY                     0
TOTAL_TRADES_SELL                    0
VOLUME                               0
VOLUME_BUY                           0
VOLUME_SELL                          0
return                               0
log_return                           0
hourly_volatility                    0
EMA_8                                0
EMA_13                               0
EMA_21                               0
EMA_Signal                           0
EMA_short                            0
EMA_long                             0
MACD                                 0
Signal_Line                          0
MACD_Signal                          0
MACD_Hist                

In [12]:
e = Engine(initial_cash = 100_000,asset_type='cryptocurrencies')
e.add_data(validate_frame)
e.add_strategy(MultiStrategyAggregator()) # Pass both df_filtered and strategy_dict
stats = e.run()

 25%|██▌       | 153/612 [00:00<00:00, 707.37it/s]

2025-01-01 03:00:00+00:00 New position opened: <Position: btc size: 0.10627116 entry: 94098.91>
2025-01-01 04:00:00+00:00 Stop Loss Filled. Limit: 93795.88416666667 / Low: 93500.0
2025-01-01 04:00:00+00:00 Position closed for ticker btc at 93795.88416666667
2025-01-01 13:00:00+00:00 New position opened: <Position: btc size: 0.10658622 entry: 93820.76>
2025-01-01 14:00:00+00:00 New position opened: <Position: btc size: 0.11700567 entry: 94031.76>
2025-01-01 15:00:00+00:00 New position opened: <Position: btc size: 0.12854348 entry: 94175.89>
2025-01-01 16:00:00+00:00 New position opened: <Position: btc size: 0.14110879 entry: 94432.94>
2025-01-01 17:00:00+00:00 Stop Loss Filled. Limit: 93886.06583333333 / Low: 93760.93
2025-01-01 17:00:00+00:00 Position closed for ticker btc at 93886.06583333333
2025-01-01 17:00:00+00:00 Stop Loss Filled. Limit: 94143.11583333333 / Low: 93760.93
2025-01-01 17:00:00+00:00 Position closed for ticker btc at 94143.11583333333
2025-01-01 17:00:00+00:00 New po

 64%|██████▎   | 389/612 [00:00<00:00, 850.20it/s]

2025-01-07 09:00:00+00:00 New position opened: <Position: btc size: 0.09813053 entry: 101905.08>
2025-01-07 10:00:00+00:00 Stop Loss Filled. Limit: 101759.90166666667 / Low: 101379.45
2025-01-07 10:00:00+00:00 Position closed for ticker btc at 101759.90166666667
2025-01-08 23:00:00+00:00 New position opened: <Position: btc size: 0.10494178 entry: 95290.93>
2025-01-09 00:00:00+00:00 Stop Loss Filled. Limit: 94870.62583333332 / Low: 94809.07
2025-01-09 00:00:00+00:00 Position closed for ticker btc at 94870.62583333332
2025-01-09 16:00:00+00:00 New position opened: <Position: btc size: 0.10610753 entry: 94244.02>
2025-01-09 17:00:00+00:00 Stop Loss Filled. Limit: 93690.94416666667 / Low: 92325.0
2025-01-09 17:00:00+00:00 Position closed for ticker btc at 93690.94416666667
2025-01-09 17:00:00+00:00 New position opened: <Position: btc size: 0.11671048 entry: 94250.95>
2025-01-09 18:00:00+00:00 Stop Loss Filled. Limit: 93588.24916666666 / Low: 92403.77
2025-01-09 18:00:00+00:00 Position clos

 93%|█████████▎| 569/612 [00:00<00:00, 839.16it/s]

2025-01-17 05:00:00+00:00 New position opened: <Position: btc size: 0.09854033 entry: 101481.29>
2025-01-17 06:00:00+00:00 New position opened: <Position: btc size: 0.10853755 entry: 101334.05>
2025-01-17 07:00:00+00:00 New position opened: <Position: btc size: 0.11931837 entry: 101408.99>
2025-01-17 08:00:00+00:00 New position opened: <Position: btc size: 0.13121958 entry: 101440.5>
2025-01-17 09:00:00+00:00 New position opened: <Position: btc size: 0.14367024 entry: 102136.01>
2025-01-17 10:00:00+00:00 Take Profit Filled. Limit: 102523.73333333332 / High: 102581.45
2025-01-17 10:00:00+00:00 Position closed for ticker btc at 102523.73333333332
2025-01-17 10:00:00+00:00 Take Profit Filled. Limit: 102376.49333333333 / High: 102581.45
2025-01-17 10:00:00+00:00 Position closed for ticker btc at 102376.49333333333
2025-01-17 10:00:00+00:00 Take Profit Filled. Limit: 102451.43333333333 / High: 102581.45
2025-01-17 10:00:00+00:00 Position closed for ticker btc at 102451.43333333333
2025-01-1

100%|██████████| 612/612 [00:00<00:00, 820.59it/s]

Metric                                   |      Value
-------------------------------------------------------
Total Return (%)                         |       5.08
Average Exposure to Asset (%)            |      19.30
Strategy CAGR (%)                        |     106.28
Buy & Hold CAGR (%)                      |     361.04
Strategy Volatility (%)                  |       3.04
Buy & Hold Volatility (%)                |      11.22
Strategy Sharpe Ratio                    |      34.92
Buy & Hold Sharpe Ratio                  |      32.17
Strategy Max Drawdown (%)                |      -1.71
Buy & Hold Max Drawdown (%)              |     -11.25
Number of Trades                         |     680.00
Number of Buys                           |     340.00
Number of Sells                          |     340.00
Number of Take Profits                   |      98.00
Number of Stop Losses                    |     242.00


In [14]:
e.plot(show_signals= False)